In [1]:
from sqlalchemy.testing.suite.test_reflection import metadata
%load_ext autoreload
%autoreload 2

In [2]:
from agents import gen_trace_id, RunResult, Runner, Agent, function_tool
from agents import trace
from dotenv import load_dotenv, find_dotenv
from openai.types.responses import EasyInputMessageParam, ResponseInputTextParam
from openai.types.responses import ResponseInputImageParam

from farmbase_client.models import Location
from farmwise.agents import triage_agent_instructions
from farmwise.context import UserContext

load_dotenv(find_dotenv())


@function_tool
def get_weather(location: str):
    return f"It's sunny and warm in {location}"


weatherman = Agent(
    name="Weatherman",
    instructions="You can look up the weather",
    handoff_description="An agent that can look up the weather for a location",
    tools=[get_weather],
    model="gpt-4.1",
)

crop_doctor = Agent(
    name="Crop disease doctor",
    instructions="You provide help with crop diseases. Analyse a photo and tell the user what is causing the disease and what a good treatment is",
    handoff_description="An agent that can identify crop pests and diseases from an image",
    handoffs=[weatherman],
    model="gpt-4.1",
)

triage_agent: Agent[UserContext] = Agent(
    name="Triage Agent",
    handoff_description="A triage agent that can delegate a customer's request to the appropriate agent.",
    instructions=triage_agent_instructions,
    handoffs=[
        crop_doctor,
        weatherman
    ],
    # output_type=WhatsappResponse,
    model="gpt-4.1",
)

weatherman.handoffs.append(triage_agent)
crop_doctor.handoffs.append(triage_agent)

input_items = [
    EasyInputMessageParam(
        content=[
            ResponseInputTextParam(
                text="What is going on in this image? Also, what's the weather like in Bungoma today?",
                type="input_text"),
            ResponseInputImageParam(
                detail="auto",
                image_url="https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcSGC3jgQd0YVICGXMor-i7mbtlplOB2oQGkCDbmVIA5QT_xek16LDJaV69zpkDlc-oYH9jb7x19c0OZ8mzLAK6BVIhxx44d6OOmhKdu6m4",
                # image_url=f"data:image/jpeg;base64,{user_input.image}",
                type="input_image"
            )
        ],
        role="user",
    )]

context = UserContext(user_id=1, name="Mark", phone_number="31657775781",
                      location=Location(latitude=1.0, longitude=1.0))


async def run_agent():
    trace_id = gen_trace_id()
    with trace("FarmWise", trace_id=trace_id, group_id="31657775781"):
        _result: RunResult = await Runner.run(
            triage_agent,
            input=input_items,
            context=context,
            # previous_response_id=chat_state.previous_response_id,
        )

    print(f"https://platform.openai.com/traces/trace?trace_id={trace_id}")
    return _result


result = await run_agent()
result

https://platform.openai.com/traces/trace?trace_id=trace_a0b4f2ed4293444b9aca6f458ed71766


RunResult(input=[{'content': [{'text': "What is going on in this image? Also, what's the weather like in Bungoma today?", 'type': 'input_text'}, {'detail': 'auto', 'image_url': 'https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcSGC3jgQd0YVICGXMor-i7mbtlplOB2oQGkCDbmVIA5QT_xek16LDJaV69zpkDlc-oYH9jb7x19c0OZ8mzLAK6BVIhxx44d6OOmhKdu6m4', 'type': 'input_image'}], 'role': 'user'}], new_items=[HandoffCallItem(agent=Agent(name='Triage Agent', instructions=<function triage_agent_instructions at 0x10a05fe20>, handoff_description="A triage agent that can delegate a customer's request to the appropriate agent.", handoffs=[Agent(name='Crop disease doctor', instructions='You provide help with crop diseases. Analyse a photo and tell the user what is causing the disease and what a good treatment is', handoff_description='An agent that can identify crop pests and diseases from an image', handoffs=[Agent(name='Weatherman', instructions='You can look up the weather', handoff_description='An agent tha

In [26]:
from farmbase.database.core import Base
from sqlalchemy import create_engine
from farmbase.messages.models import RunResult
from sqlalchemy.orm import Session

engine = create_engine("sqlite:///sqlalchemy_play.db", echo=True)
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

# conn = engine.connect()

2025-05-13 14:33:42,502 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-13 14:33:42,503 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("agent")
2025-05-13 14:33:42,504 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-13 14:33:42,505 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("run_result")
2025-05-13 14:33:42,505 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-13 14:33:42,506 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("model_response")
2025-05-13 14:33:42,506 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-13 14:33:42,507 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("run_item")
2025-05-13 14:33:42,507 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-13 14:33:42,508 INFO sqlalchemy.engine.Engine 
DROP TABLE run_item
2025-05-13 14:33:42,509 INFO sqlalchemy.engine.Engine [no key 0.00029s] ()
2025-05-13 14:33:42,510 INFO sqlalchemy.engine.Engine 
DROP TABLE model_response
2025-05-13 14:33:42,511 INFO sqlalchemy.engine.Engine [no key 0.00035

In [27]:
from farmbase.messages import models
with Session(engine) as session:
    crop_doctor_model = models.Agent(name=crop_doctor.name)
    triage_model = models.Agent(name=triage_agent.name)
    weatherman_model = models.Agent(name=weatherman.name)
    # session.reload()
    session.add_all([crop_doctor_model, triage_model, weatherman_model])
    session.commit()

    agents_dict = {
        crop_doctor.name: crop_doctor_model,
        triage_model.name: triage_model,
        weatherman_model.name: weatherman_model,
    }



2025-05-13 14:33:43,581 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-13 14:33:43,583 INFO sqlalchemy.engine.Engine INSERT INTO agent (name, metadata) VALUES (?, ?) RETURNING id
2025-05-13 14:33:43,584 INFO sqlalchemy.engine.Engine [generated in 0.00013s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('Crop disease doctor', '{}')
2025-05-13 14:33:43,584 INFO sqlalchemy.engine.Engine INSERT INTO agent (name, metadata) VALUES (?, ?) RETURNING id
2025-05-13 14:33:43,585 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/3 (ordered; batch not supported)] ('Triage Agent', '{}')
2025-05-13 14:33:43,585 INFO sqlalchemy.engine.Engine INSERT INTO agent (name, metadata) VALUES (?, ?) RETURNING id
2025-05-13 14:33:43,586 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/3 (ordered; batch not supported)] ('Weatherman', '{}')
2025-05-13 14:33:43,587 INFO sqlalchemy.engine.Engine COMMIT
2025-05-13 14:33:43,588 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-13 14:33:43,589 INF

In [29]:


with Session(engine) as session:
    res = models.RunResult(
        input_=result.input,
        final_output=result.final_output,
        input_guardrails=None,
        output_guardrails=None,
        last_agent=agents_dict[result.last_agent.name],
        raw_responses=[
            models.ModelResponse(
                response_id=r.response_id,
                output=[[r.model_dump() for r in raw_resp.output] for raw_resp in result.raw_responses],
                usage=[asdict(raw_resp.usage) for raw_resp in result.raw_responses]
            ) for r in result.raw_responses],
        new_items=[get_run_item_model(i) for i in result.new_items],
        input_list=result.to_input_list(),
    )
    session.add(res)
    session.commit()
    # session.refresh()

2025-05-13 14:33:51,666 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-13 14:33:51,668 INFO sqlalchemy.engine.Engine SELECT agent.id AS agent_id, agent.name AS agent_name, agent.metadata AS agent_metadata 
FROM agent 
WHERE agent.id = ?
2025-05-13 14:33:51,669 INFO sqlalchemy.engine.Engine [cached since 6.252s ago] (3,)
2025-05-13 14:33:51,670 INFO sqlalchemy.engine.Engine SELECT agent.id AS agent_id, agent.name AS agent_name, agent.metadata AS agent_metadata 
FROM agent 
WHERE agent.id = ?
2025-05-13 14:33:51,671 INFO sqlalchemy.engine.Engine [cached since 6.254s ago] (2,)
2025-05-13 14:33:51,671 INFO sqlalchemy.engine.Engine SELECT agent.id AS agent_id, agent.name AS agent_name, agent.metadata AS agent_metadata 
FROM agent 
WHERE agent.id = ?
2025-05-13 14:33:51,672 INFO sqlalchemy.engine.Engine [cached since 6.256s ago] (1,)
2025-05-13 14:33:51,674 INFO sqlalchemy.engine.Engine INSERT INTO run_result (created_at, input, final_output, input_guardrails, output_guardrails, last

In [30]:
agents_dict

{'Crop disease doctor': <Agent #1>,
 'Triage Agent': <Agent #2>,
 'Weatherman': <Agent #3>}

In [48]:

from datetime import UTC
from datetime import datetime
from farmwise.items import get_run_item_model
from farmbase_client.models import RunResultCreate, AgentBase
from farmbase_client import AuthenticatedClient
from farmbase_client.api.messages import messages_create_run_result

token = "fsdfds"


#     res = models.RunResult(
#         input_=result.input,
#         final_output=result.final_output,
#         input_guardrails=None,
#         output_guardrails=None,
#         last_agent=agents_dict[result.last_agent.name],
#         raw_responses=[
#             models.ModelResponse(
#                 response_id=r.response_id,
#                 output=[[r.model_dump() for r in raw_resp.output] for raw_resp in result.raw_responses],
#                 usage=[asdict(raw_resp.usage) for raw_resp in result.raw_responses]
#             ) for r in result.raw_responses],
#         new_items=[get_run_item_model(i) for i in result.new_items],
#         input_list=result.to_input_list(),
#     )



with AuthenticatedClient(base_url="http://127.0.0.1:8000/api/v1", token=token) as client:
    response = await messages_create_run_result.asyncio(
        client=client, organization='default',
        body=RunResultCreate(
            contact_id=1,
            created_at=datetime.now(UTC),
            input=result.input,
            final_output=result.final_output,
            input_guardrails=None,
            output_guardrails=None,
            last_agent=AgentBase(
                name=result.last_agent.name
            ),
            # new_items=[get_run_item_model(i) for i in result.new_items],
            new_items=[],
            raw_responses=[],
            input_list=result.to_input_list()
        ))
    print(response)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [53]:
from farmbase_client.api.messages import messages_get_input_list

with AuthenticatedClient(base_url="http://127.0.0.1:8000/api/v1", token=token) as client:
    response = await messages_get_input_list.asyncio(client=client, organization='default', contact_id=1)
    print(response)

None
